# Descarga de Resoluciones de Calificación Ambiental

Este notebook automatiza el proceso de descarga de las Resoluciones de Calificación Ambiental (RCA). A partir de un archivo Excel que contiene las URLs de descarga, se accede a cada página, se descarga el documento PDF y se extrae el número de resolución. 

## Importar Librerías Necesarias

Se importan únicamente las librerías requeridas, organizadas por su funcionalidad.

In [ ]:
# Librerías del sistema y manipulación de archivos
import os
import shutil
import time
import psutil

# Librerías para manejo de datos
import pandas as pd
import openpyxl

# Librerías para automatización y webscrapping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import pyautogui

## Configuración de Directorios

Se definen las rutas de origen (descargas) y destino, utilizando la variable de entorno `USERPROFILE` para mayor portabilidad.

In [ ]:
sector = "energia"
origen = os.path.join(os.environ['USERPROFILE'], "RUTA")
destino = os.path.join(os.environ['USERPROFILE'], "RUTA", sector)

os.makedirs(destino, exist_ok=True)

## Funciones Auxiliares

Se definen las funciones de apoyo para:

- Extraer hipervínculos del Excel.
- Esperar la descarga de archivos.
- Cerrar visores de PDF.
- Extraer el número de la RCA de la página web.

In [ ]:
def extract_hyperlinks(sheet):
    hyperlinks = {}
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                hyperlinks[cell.coordinate] = cell.hyperlink.target
    return hyperlinks

def wait_for_download(download_dir):
    timeout = 30
    start_time = time.time()
    while True:
        time.sleep(1)
        files = os.listdir(download_dir)
        pdf_files = [file for file in files if file.endswith('.pdf')]
        if pdf_files:
            return pdf_files[0]
        if time.time() - start_time > timeout:
            return False

def close_pdf_viewers():
    pdf_viewers = ["Acrobat.exe", "AcroRd32.exe", "FoxitReader.exe", "SumatraPDF.exe", "Evince.exe", "pdfviewer.exe"]
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if proc.info['name'] in pdf_viewers:
                proc.terminate()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

def extract_rca_number(driver, wait):
    estados_table = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//td[contains(text(), 'Registro de Estados del Proyecto')]/following-sibling::td//table")
    ))
    rows = estados_table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) >= 3:
            if "Aprobado" in cells[0].text and "RCA" in cells[1].text:
                return cells[2].text
    return None

## Función Unificada para Procesar RCA

Esta función abre la URL, descarga el PDF de la RCA, extrae el número de resolución y mueve el archivo descargado al directorio de destino. Se utiliza un único navegador para realizar ambas tareas y se implementa un manejo básico de errores.

In [ ]:
def process_rca(url, cell, origen, destino, options):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    downloaded_file = False
    try:
        driver.get(url)
        time.sleep(2)  

        try:
            download_link = driver.find_element(By.LINK_TEXT, "Resolución de calificación ambiental (RCA)")
            download_link.click()
        except:
            try:
                download_link = driver.find_element(By.LINK_TEXT, "Resolución de Calificación Ambiental (RCA)")
                download_link.click()
            except:
                download_link = driver.find_element(By.XPATH, "//a[contains(@title, 'Documento digital')]")
                download_link.click()
        time.sleep(5)  
        
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
        
        continue_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
        ))
        continue_button.click()
        time.sleep(5)
        
        downloaded_file = wait_for_download(origen)
        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(10)
    except Exception as e:

        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(2)
            if not downloaded_file:
                try:
                    print_button = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, "//a[contains(@class, 'button2') and contains(text(), 'IMPRIMIR')]")
                    ))
                    print_button.click()
                    time.sleep(5)
                    pyautogui.moveTo(140,300)
                    pyautogui.click()
                    pyautogui.press('tab')
                    pyautogui.press('enter')
                    time.sleep(5)
                    pyautogui.press('down')
                    pyautogui.press('enter', presses=2)
                    time.sleep(5)
                    pyautogui.press('tab', presses=4)
                    pyautogui.press('enter')
                    time.sleep(5)
                    pyautogui.press('enter')
                except:
                    pass
                downloaded_file = wait_for_download(origen)
    finally:

        driver.quit()
        close_pdf_viewers()
        
        if downloaded_file:
            try:
                for filename in os.listdir(origen):
                    archivo_origen = os.path.join(origen, filename)
                    archivo_destino = os.path.join(destino, f"{cell}.pdf")
                    if os.path.isfile(archivo_origen):
                        shutil.move(archivo_origen, archivo_destino)
            except Exception as e:
                print(f"No se pudo mover el archivo: {e}")
    return 

## Proceso Principal: Lectura del Excel y Ejecución Unificada

Se carga el archivo Excel con las URLs, se extraen los hipervínculos y, para cada registro, se ejecuta el proceso unificado de descarga y extracción del número de RCA.

In [ ]:
# Ruta del archivo Excel 
file_path = os.path.join(
    os.environ['USERPROFILE'], 
    "RUTA", f"archivos_{sector}", f"seia_{sector}.xlsx"
)
df = pd.read_excel(file_path, engine='openpyxl')

In [ ]:
# Abrir el archivo Excel con openpyxl para obtener los hipervínculos
workbook = openpyxl.load_workbook(file_path, data_only=True)
sheet = workbook.active

# Función para extraer hipervínculos de una celda
def extract_hyperlinks(sheet):
    hyperlinks = {}
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                hyperlinks[cell.coordinate] = cell.hyperlink.target
    return hyperlinks

# Obtener los hipervínculos del archivo
hyperlinks = extract_hyperlinks(sheet)

# Mostrar los hipervínculos encontrados
for cell, link in hyperlinks.items():
    print(f"Cell {cell}: {link}")

# Opcional: agregar los hipervínculos al DataFrame
df_hyperlinks = pd.DataFrame(list(hyperlinks.items()), columns=['cell', 'hyperlink'])
print(df_hyperlinks)

for i, (cell, hyperlink) in enumerate(hyperlinks.items()):
    if i < len(df):
        df.at[i, "cell"] = cell
        df.at[i, "hyperlink"] = hyperlink

In [ ]:

options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": origen,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-extensions")
## options.add_argument("--headless")  # Descomentar para ejecución en segundo plano
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

### Nuevo Código para Nueva Página Web

In [ ]:
def extract_rca_number(driver, wait):
    """
    Función actualizada para extraer el número de RCA de la nueva estructura de la página
    """
    try:
        # Buscar la tabla con el nuevo ID
        estados_table = wait.until(EC.presence_of_element_located(
            (By.ID, "detallelistado")
        ))
        
        # Buscar todas las filas del tbody
        rows = estados_table.find_elements(By.XPATH, ".//tbody/tr")
        
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                estado = cells[0].text.strip()
                documento = cells[1].text.strip()
                numero = cells[2].text.strip()
                
                # Buscar la fila que contenga "Aprobado" y "RCA"
                if "Aprobado" in estado and "RCA" in documento:
                    return numero
        
        # Si no encontramos con el criterio anterior, buscar cualquier RCA
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                documento = cells[1].text.strip()
                numero = cells[2].text.strip()
                
                if "RCA" in documento or "Resolución de Calificación Ambiental" in documento:
                    return numero
                    
    except Exception as e:
        print(f"Error al extraer número RCA: {e}")
        
        # Método alternativo: buscar directamente por texto del enlace
        try:
            rca_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Resolución de Calificación Ambiental')]")
            # Buscar el número en la misma fila
            row = rca_link.find_element(By.XPATH, "./ancestor::tr")
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                return cells[2].text.strip()
        except:
            pass
    
    return None

# También necesitas actualizar el código principal para manejar mejor los errores:
def process_rca_with_number_extraction(url, cell, origen, destino, options):
    """
    Función combinada que procesa la descarga y extrae el número RCA
    """
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 5)  # Aumentar timeout
    downloaded_file = False
    rca_number = None

    try:
        driver.get(url)
        time.sleep(2)  # Esperar carga inicial
        
        # Primero extraer el número RCA
        rca_number = extract_rca_number(driver, wait)
        print(f"RCA Number extracted for {cell}: {rca_number}")
        
        # Luego proceder con la descarga
        try:
            download_link = driver.find_element(By.LINK_TEXT, "Resolución de calificación ambiental (RCA)")
            download_link.click()
        except:
            try:
                download_link = driver.find_element(By.LINK_TEXT, "Resolución de Calificación Ambiental (RCA)")
                download_link.click()
            except:
                download_link = driver.find_element(By.XPATH, "//a[contains(@title, 'Documento digital')]")
                download_link.click()

        time.sleep(2)  

        # Cambiar a la nueva pestaña si existe
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])

        # Hacer clic en el botón "Continuar"
        continue_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
        ))
        continue_button.click()
        time.sleep(5)

        # Esperar la descarga del archivo
        downloaded_file = wait_for_download(origen)
        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(4)

    except Exception as e:
        print(f"Error en proceso principal: {e}")
        # Método alternativo similar al original
        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(2)
            if not downloaded_file:
                print(f"Error en proceso principal 2: {e}")
                try:
                    print_button = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, "//a[contains(@class, 'button2') and contains(text(), 'IMPRIMIR')]")
                    ))
                    print_button.click()
                    time.sleep(5)
                    # ... resto del código de automatización con pyautogui
                except:
                    pass
                downloaded_file = wait_for_download(origen)

    finally:
        driver.quit()
        close_pdf_viewers()
        
        # Mover el archivo descargado al directorio de destino
        if downloaded_file:
            try:
                for filename in os.listdir(origen):
                    archivo_origen = os.path.join(origen, filename)
                    archivo_destino = os.path.join(destino, f"{cell}.pdf")
                    if os.path.isfile(archivo_origen):
                        shutil.move(archivo_origen, archivo_destino)
            except Exception as e:
                print(f"No se pudo mover el archivo: {e}")

    return rca_number

# Código principal actualizado
for index, row in df[:].iterrows():
    cell = row["cell"]
    url = row["hyperlink"]
    
    # Procesar descarga y extraer número en una sola función
    rca_number = process_rca_with_number_extraction(url, cell, origen, destino, options)
    
    print(f"RCA Number for {cell}: {rca_number}")
    df.loc[index, "numero"] = rca_number

In [ ]:
def extract_rca_number(driver, wait):
    """
    Función actualizada para extraer el número de RCA de la nueva estructura de la página
    """
    try:
        # Buscar la tabla con el nuevo ID
        estados_table = wait.until(EC.presence_of_element_located(
            (By.ID, "detallelistado")
        ))
        
        # Buscar todas las filas del tbody
        rows = estados_table.find_elements(By.XPATH, ".//tbody/tr")
        
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                estado = cells[0].text.strip()
                documento = cells[1].text.strip()
                numero = cells[2].text.strip()
                
                # Buscar la fila que contenga "Aprobado" y "RCA"
                if "Aprobado" in estado and "RCA" in documento:
                    return numero
        
        # Si no encontramos con el criterio anterior, buscar cualquier RCA
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                documento = cells[1].text.strip()
                numero = cells[2].text.strip()
                
                if "RCA" in documento or "Resolución de Calificación Ambiental" in documento:
                    return numero
                    
    except Exception as e:
        print(f"Error al extraer número RCA: {e}")
        
        # Método alternativo: buscar directamente por texto del enlace
        try:
            rca_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Resolución de Calificación Ambiental')]")
            # Buscar el número en la misma fila
            row = rca_link.find_element(By.XPATH, "./ancestor::tr")
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 3:
                return cells[2].text.strip()
        except:
            pass
    
    return None

# También necesitas actualizar el código principal para manejar mejor los errores:
def process_rca_with_number_extraction(url, cell, origen, destino, options):
    """
    Función combinada que procesa la descarga y extrae el número RCA
    """
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 5)  # Aumentar timeout
    downloaded_file = False
    rca_number = None

    try:
        driver.get(url)
        time.sleep(3)  # Esperar carga inicial
        
        # Primero extraer el número RCA
        rca_number = extract_rca_number(driver, wait)
        print(f"RCA Number extracted for {cell}: {rca_number}")
        
        # Luego proceder con la descarga
        try:
            download_link = driver.find_element(By.LINK_TEXT, "Resolución de calificación ambiental (RCA)")
            download_link.click()
        except:
            try:
                download_link = driver.find_element(By.LINK_TEXT, "Resolución de Calificación Ambiental (RCA)")
                download_link.click()
            except:
                download_link = driver.find_element(By.XPATH, "//a[contains(@title, 'Documento digital')]")
                download_link.click()

        time.sleep(5)  

        # Cambiar a la nueva pestaña si existe
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])

        # Hacer clic en el botón "Continuar"
        continue_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
        ))
        continue_button.click()
        time.sleep(2)

        # Esperar la descarga del archivo
        downloaded_file = wait_for_download(origen)
        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(2)

    except Exception as e:
        print(f"Error en proceso principal: {e}")
        # Método alternativo similar al original
        while not downloaded_file:
            try:
                download_button = wait.until(EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class, 'btn-download') and contains(@href, 'documentDownload')]")
                ))
                download_button.click()
            except:
                pass
            downloaded_file = wait_for_download(origen)
            time.sleep(2)
            if not downloaded_file:
                try:
                    print_button = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, "//a[contains(@class, 'button2') and contains(text(), 'IMPRIMIR')]")
                    ))
                    print_button.click()
                    time.sleep(5)
                    # ... resto del código de automatización con pyautogui
                except:
                    pass
                downloaded_file = wait_for_download(origen)

    finally:
        driver.quit()
        close_pdf_viewers()
        
        # Mover el archivo descargado al directorio de destino
        if downloaded_file:
            try:
                for filename in os.listdir(origen):
                    archivo_origen = os.path.join(origen, filename)
                    archivo_destino = os.path.join(destino, f"{cell}.pdf")
                    if os.path.isfile(archivo_origen):
                        shutil.move(archivo_origen, archivo_destino)
            except Exception as e:
                print(f"No se pudo mover el archivo: {e}")

    return rca_number

# Código principal actualizado
for index, row in df[:].iterrows():
    cell = row["cell"]
    url = row["hyperlink"]
    
    # Procesar descarga y extraer número en una sola función
    rca_number = process_rca_with_number_extraction(url, cell, origen, destino, options)
    
    print(f"RCA Number for {cell}: {rca_number}")
    df.loc[index, "numero"] = rca_number

In [ ]:
def process_rca_with_number_extraction(url, cell, origen, destino, options):
    """
    Función combinada que procesa la descarga y extrae el número RCA
    """
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 5)  # Aumentar timeout
    downloaded_file = False
    rca_number = None

    try:
        driver.get(url)
        time.sleep(2)  # Esperar carga inicial más tiempo
        
        # Primero extraer el número RCA
        rca_number = extract_rca_number(driver, wait)
        print(f"RCA Number extracted for {cell}: {rca_number}")
        
        # Cerrar cualquier modal o overlay que pueda estar abierto
        try:
            close_button = driver.find_element(By.XPATH, "//button[contains(@class, 'close') or contains(@data-dismiss, 'modal')]")
            close_button.click()
            time.sleep(1)
        except:
            pass
        
        # Scroll hasta el elemento antes de hacer click
        def safe_click_rca_link(driver, wait):
            """Función para hacer click seguro en el link RCA"""
            attempts = [
                "//a[contains(text(), 'Resolución de calificación ambiental (RCA)')]",
                "//a[contains(text(), 'Resolución de Calificación Ambiental (RCA)')]",
                "//a[contains(text(), 'Resolucion de Calificacion Ambiental (RCA)')]",
                "//a[contains(text(), 'Resolucion de Calificación Ambiental (RCA)')]",
                "//a[contains(text(), 'Resolucion de calificacion Ambiental (RCA)')]",
                "//a[contains(text(), 'RCA')]",
                "//a[contains(@href, 'documentos') and contains(text(), 'Resolución')]"
            ]
            
            for attempt_xpath in attempts:
                try:
                    # Buscar el elemento
                    download_link = wait.until(EC.presence_of_element_located((By.XPATH, attempt_xpath)))
                    
                    # Scroll hasta el elemento
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", download_link)
                    time.sleep(2)
                    
                    # Intentar click normal
                    try:
                        # Esperar a que sea clickeable
                        clickable_link = wait.until(EC.element_to_be_clickable((By.XPATH, attempt_xpath)))
                        clickable_link.click()
                        return True
                    except Exception as e:
                        print(f"Click normal falló: {e}")
                        
                        # Intentar con JavaScript
                        try:
                            driver.execute_script("arguments[0].click();", download_link)
                            return True
                        except Exception as js_e:
                            print(f"Click JavaScript falló: {js_e}")
                            
                            # Intentar con ActionChains
                            try:
                                actions = ActionChains(driver)
                                actions.move_to_element(download_link).click().perform()
                                return True
                            except Exception as action_e:
                                print(f"ActionChains click falló: {action_e}")
                                continue
                                
                except Exception as find_e:
                    print(f"No se encontró elemento con xpath {attempt_xpath}: {find_e}")
                    continue
            
            return False
        
        # Intentar hacer click en el link RCA
        if not safe_click_rca_link(driver, wait):
            raise Exception("No se pudo hacer click en ningún link RCA")
            
        time.sleep(2)  

        # Cambiar a la nueva pestaña si existe
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)

        # Hacer clic en el botón "Continuar"
        try:
            continue_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
            ))
            # Scroll al botón antes de hacer click
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", continue_button)
            time.sleep(1)
            continue_button.click()
        except:
            # Intentar con JavaScript si falla
            try:
                continue_button = driver.find_element(By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
                driver.execute_script("arguments[0].click();", continue_button)
            except:
                pass
                
        time.sleep(5)

        # Esperar la descarga del archivo
        downloaded_file = wait_for_download(origen)

    except Exception as e:
        print(f"Error en proceso principal: {e}")


    finally:
        driver.quit()
        close_pdf_viewers()

        try:
            for filename in os.listdir(origen):
                archivo_origen = os.path.join(origen, filename)
                archivo_destino = os.path.join(destino, f"{cell}.pdf")
                if os.path.isfile(archivo_origen):
                    shutil.move(archivo_origen, archivo_destino)
                    print(f"Archivo movido exitosamente: {cell}.pdf")
                    break
        except Exception as e:
            print(f"No se pudo mover el archivo: {e}")

    return rca_number


In [ ]:
# Código principal actualizado
for index, row in df[:].iterrows():
    cell = row["cell"]
    url = row["hyperlink"]
    
    # Procesar descarga y extraer número en una sola función
    rca_number = process_rca_with_number_extraction(url, cell, origen, destino, options)
    
    print(f"RCA Number for {cell}: {rca_number}")
    df.loc[index, "numero"] = rca_number

In [ ]:
def safe_click_rca_link(driver, wait):
    """Función para hacer click seguro en el link RCA"""
    attempts = [
        # Buscar por el texto exacto que aparece en el HTML
        "//a[contains(text(), 'Resolucion de Calificación Ambiental (RCA)')]",
        # Versión con tilde por si acaso
        "//a[contains(text(), 'Resolución de Calificación Ambiental (RCA)')]",
        # Buscar en la celda td-primary
        "//td[@class='td-primary']//a[contains(text(), 'Resolucion')]",
        "//td[@class='td-primary']//a[contains(text(), 'Resolución')]",
        # Buscar por href que contenga .pdf
        "//a[contains(@href, '.pdf') and contains(text(), 'Resolucion')]",
        "//a[contains(@href, '.pdf') and contains(text(), 'RCA')]",
        # Buscar en la tabla por cualquier link que contenga RCA
        "//table[@id='detallelistado']//a[contains(text(), 'RCA')]",
        # Buscar por texto más genérico
        "//a[contains(text(), 'RCA')]",
        # Buscar cualquier enlace en la columna de documento
        "//tbody//tr//td[2]//a"
    ]
    
    for i, attempt_xpath in enumerate(attempts):
        try:
            print(f"Intento {i+1}: Buscando con xpath: {attempt_xpath}")
            
            # Buscar el elemento
            download_link = wait.until(EC.presence_of_element_located((By.XPATH, attempt_xpath)))
            print(f"Elemento encontrado: {download_link.text}")
            
            # Scroll hasta el elemento
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", download_link)
            time.sleep(2)
            
            # Intentar click normal
            try:
                # Esperar a que sea clickeable
                clickable_link = wait.until(EC.element_to_be_clickable((By.XPATH, attempt_xpath)))
                clickable_link.click()
                print("Click normal exitoso")
                return True
            except Exception as e:
                print(f"Click normal falló: {e}")
                
                # Intentar con JavaScript
                try:
                    driver.execute_script("arguments[0].click();", download_link)
                    print("Click JavaScript exitoso")
                    return True
                except Exception as js_e:
                    print(f"Click JavaScript falló: {js_e}")
                    
                    # Intentar con ActionChains
                    try:
                        actions = ActionChains(driver)
                        actions.move_to_element(download_link).click().perform()
                        print("Click ActionChains exitoso")
                        return True
                    except Exception as action_e:
                        print(f"ActionChains click falló: {action_e}")
                        
                        # Último intento: navegar directamente al href
                        try:
                            href = download_link.get_attribute('href')
                            if href:
                                print(f"Navegando directamente a: {href}")
                                driver.get(href)
                                return True
                        except Exception as href_e:
                            print(f"Navegación directa falló: {href_e}")
                            continue
                            
        except Exception as find_e:
            print(f"No se encontró elemento con xpath {attempt_xpath}: {find_e}")
            continue
    
    # Si todos los intentos fallan, intentar buscar todos los enlaces y mostrar información
    try:
        print("Buscando todos los enlaces disponibles...")
        all_links = driver.find_elements(By.TAG_NAME, "a")
        for link in all_links:
            text = link.text.strip()
            href = link.get_attribute('href')
            if text and ('RCA' in text.upper() or 'RESOLUCION' in text.upper() or 'CALIFICACION' in text.upper()):
                print(f"Enlace encontrado - Texto: '{text}' | Href: '{href}'")
    except Exception as debug_e:
        print(f"Error en debug: {debug_e}")
    
    return False

# También agrega esta función auxiliar para debugging
def debug_page_content(driver):
    """Función para debuggear el contenido de la página"""
    try:
        # Buscar la tabla específica
        table = driver.find_element(By.ID, "detallelistado")
        rows = table.find_elements(By.XPATH, ".//tbody//tr")
        
        print(f"Encontradas {len(rows)} filas en la tabla")
        
        for i, row in enumerate(rows):
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 2:
                document_cell = cells[1]  # La segunda celda contiene el documento
                links = document_cell.find_elements(By.TAG_NAME, "a")
                for link in links:
                    text = link.text.strip()
                    href = link.get_attribute('href')
                    print(f"Fila {i+1} - Enlace: '{text}' | Href: '{href}'")
                    
    except Exception as e:
        print(f"Error en debug_page_content: {e}")

# Función principal actualizada
def process_rca_with_number_extraction(url, cell, origen, destino, options):
    """
    Función combinada que procesa la descarga y extrae el número RCA
    """
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)  # Aumentar timeout
    downloaded_file = False
    rca_number = None

    try:
        driver.get(url)
        time.sleep(3)  # Esperar carga inicial más tiempo
        
        # Primero extraer el número RCA
        rca_number = extract_rca_number(driver, wait)
        print(f"RCA Number extracted for {cell}: {rca_number}")
        
        # Debug: mostrar contenido de la página
        print("=== DEBUG: Contenido de la página ===")
        debug_page_content(driver)
        
        # Cerrar cualquier modal o overlay que pueda estar abierto
        try:
            close_button = driver.find_element(By.XPATH, "//button[contains(@class, 'close') or contains(@data-dismiss, 'modal')]")
            close_button.click()
            time.sleep(1)
        except:
            pass
        
        # Intentar hacer click en el link RCA
        print("=== Iniciando búsqueda del enlace RCA ===")
        if not safe_click_rca_link(driver, wait):
            raise Exception("No se pudo hacer click en ningún link RCA")
            
        time.sleep(3)  

        # Cambiar a la nueva pestaña si existe
        if len(driver.window_handles) > 1:
            print("Cambiando a nueva pestaña...")
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)

        # Hacer clic en el botón "Continuar"
        try:
            continue_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
            ))
            # Scroll al botón antes de hacer click
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", continue_button)
            time.sleep(1)
            continue_button.click()
            print("Botón Continuar clickeado")
        except:
            # Intentar con JavaScript si falla
            try:
                continue_button = driver.find_element(By.XPATH, "//button[contains(@class, 'btn-style') and contains(text(), 'Continuar')]")
                driver.execute_script("arguments[0].click();", continue_button)
                print("Botón Continuar clickeado con JavaScript")
            except:
                print("No se encontró botón Continuar")
                pass
                
        time.sleep(5)

        # Esperar la descarga del archivo
        downloaded_file = wait_for_download(origen)

    except Exception as e:
        print(f"Error en proceso principal: {e}")

    finally:
        driver.quit()
        close_pdf_viewers()

        try:
            for filename in os.listdir(origen):
                archivo_origen = os.path.join(origen, filename)
                archivo_destino = os.path.join(destino, f"{cell}.pdf")
                if os.path.isfile(archivo_origen):
                    shutil.move(archivo_origen, archivo_destino)
                    print(f"Archivo movido exitosamente: {cell}.pdf")
                    break
        except Exception as e:
            print(f"No se pudo mover el archivo: {e}")

    return rca_number